# FA Score recomputation from ratios_non_null.csv

This notebook:
- Loads `ratios_non_null.csv`
- Drops `DebtToEquityRatio`
- Removes tickers with >= 2 null ratios in any latest period
- Recomputes FA Score and ranking



In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

csv_path = Path('FA_bare_minimum_selection.csv')
df = pd.read_csv(csv_path)
print('Loaded rows:', len(df))
print('Columns:', list(df.columns))

# Drop DebtToEquityRatio
cols = [c for c in df.columns if c != 'DebtToEquityRatio']
df = df[cols]

# Determine latest period per ticker by (year, quarter)
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')
df['quarter'] = pd.to_numeric(df['quarter'], errors='coerce').astype('Int64')

# Keep only latest row per ticker
latest_idx = df.groupby('ticker')[['year','quarter']].idxmax()
# idxmax on two columns returns a DataFrame; pick year first then quarter within year
latest_year_idx = df.groupby('ticker')['year'].idxmax()
latest_q_idx = df.loc[latest_year_idx].groupby('ticker')['quarter'].idxmax()
latest = df.loc[latest_q_idx].copy()

# Count null ratios per row excluding id/time columns
exclude_cols = {'ticker','year','quarter'}
ratio_cols = [c for c in latest.columns if c not in exclude_cols]
null_counts = latest[ratio_cols].isna().sum(axis=1)
filtered = latest.loc[null_counts < 2].copy()

print('Tickers before:', latest['ticker'].nunique(), 'after filter:', filtered['ticker'].nunique())
filtered.head()


Loaded rows: 5823
Columns: ['ticker', 'year', 'quarter', 'DebtToEquityRatio', 'EBITMargin', 'ROA', 'ROE', 'BasicEPS', 'PriceToBook', 'PriceToEarning', 'ROIC', 'NetRevenueGrowthYoY', 'GrossProfitGrowthYoY']


Tickers before: 1579 after filter: 1487


,ticker,year,quarter,EBITMargin,ROA,ROE,BasicEPS,PriceToBook,PriceToEarning,ROIC,NetRevenueGrowthYoY,GrossProfitGrowthYoY
2,A32,2024,5,0.033592,0.078432,0.177220,5774.870518,1.091474,6.147324,0.147334,0.010479,-0.129345
6,AAA,2025,2,0.034347,0.020683,0.050335,692.106885,0.591322,11.732292,0.047301,0.103874,0.221944
14,AAM,2025,2,0.041827,-0.001310,-0.001372,-25.934792,0.377170,-276.462599,-0.018407,0.513316,10.510410
18,AAS,2025,2,0.643070,0.014396,0.026069,284.769210,1.940792,75.303085,0.042461,-0.210153,0.291568
22,AAT,2025,2,0.054029,-0.000122,-0.000225,-2.259964,0.354763,-1584.095707,-0.002802,-0.159575,0.274628


In [6]:
# Define category mappings from available columns
# Adjust if needed to match your previous FA scoring
# Move ROIC to Financial Strength as a proxy for capital efficiency/solvency
profit_cols = [
    'ROE','ROA','EBITMargin','BasicEPS'
]

growth_cols = [
    'NetRevenueGrowthYoY','GrossProfitGrowthYoY'
]

# Financial strength: use ROIC as proxy (DebtToEquityRatio was dropped)
financial_strength_cols = ['ROIC']

valuation_cols = [
    'PriceToEarning','PriceToBook'
]

for lst in [profit_cols, growth_cols, financial_strength_cols, valuation_cols]:
    for c in list(lst):
        if c not in filtered.columns:
            lst.remove(c)

# Percentile-rank each metric so they are comparable, handling sign properly where lower is better
# For valuation: lower P/E and P/B are better, so we invert.

def percentile_rank(series: pd.Series, higher_is_better: bool = True) -> pd.Series:
    s = series.copy()
    order = s.rank(pct=True)  # ascending by default
    if higher_is_better:
        # higher raw -> higher percentile
        return order
    else:
        # lower raw -> higher percentile
        return 1 - order

work = filtered.copy()

# Build normalized columns
norm = {}
for c in profit_cols:
    norm[f'PCT_{c}'] = percentile_rank(work[c], higher_is_better=True)
for c in growth_cols:
    norm[f'PCT_{c}'] = percentile_rank(work[c], higher_is_better=True)
for c in financial_strength_cols:
    norm[f'PCT_{c}'] = percentile_rank(work[c], higher_is_better=True)
for c in valuation_cols:
    norm[f'PCT_{c}'] = percentile_rank(work[c], higher_is_better=False)

norm_df = pd.DataFrame(norm, index=work.index)

# Category scores: simple average of available normalized metrics scaled to 0-35/25/25/15 similar proportions
# Based on your FA_Score_Rankings.csv, approximate weights observed: Profitability~33, Growth~25, FS~22, Valuation~10.
# We'll use: PROFITABILITY 35, GROWTH 25, FINANCIAL_STRENGTH 25, VALUATION 15 to sum to 100.

def category_score(cols: list[str], weight: float) -> pd.Series:
    if not cols:
        return pd.Series(0.0, index=work.index)
    vals = norm_df[[f'PCT_{c}' for c in cols if f'PCT_{c}' in norm_df.columns]]
    # average ignoring NaNs
    avg = vals.mean(axis=1)
    return avg * weight

work['PROFITABILITY_Score'] = category_score(profit_cols, 35.0)
work['GROWTH_Score'] = category_score(growth_cols, 25.0)
work['FINANCIAL_STRENGTH_Score'] = category_score(financial_strength_cols, 25.0)
work['VALUATION_Score'] = category_score(valuation_cols, 15.0)

work['FA_Score'] = work[['PROFITABILITY_Score','GROWTH_Score','FINANCIAL_STRENGTH_Score','VALUATION_Score']].sum(axis=1)

# Percentile and rating mapping similar to prior outputs
work['FA_Percentile'] = work['FA_Score'].rank(pct=True) * 100

# Rating bands (example): A+ >= 90, A 80-90, B 70-80, C 60-70, D 50-60, F <50
# Adjust if you use a different scale previously
bins = [0, 50, 60, 70, 80, 90, 100]
labels = ['F','D','C','B','A','A+']
work['FA_Rating'] = pd.cut(work['FA_Percentile'], bins=bins, labels=labels, include_lowest=True, right=False)

# Rank descending by FA_Score
work = work.sort_values('FA_Score', ascending=False).reset_index(drop=True)
work['Rank'] = np.arange(1, len(work)+1)

cols_out = ['Rank','ticker','year','quarter','FA_Score','FA_Percentile','FA_Rating',
            'PROFITABILITY_Score','GROWTH_Score','FINANCIAL_STRENGTH_Score','VALUATION_Score']
result = work[cols_out]

result.head(10)


,Rank,ticker,year,quarter,FA_Score,FA_Percentile,FA_Rating,PROFITABILITY_Score,GROWTH_Score,FINANCIAL_STRENGTH_Score,VALUATION_Score
0,1,BOT,2025,2,93.923489,100.000000,NaN,33.275843,24.781258,23.317631,12.548756
1,2,VNI,2024,5,91.314752,99.932751,A+,31.823339,24.915876,23.822342,10.753194
2,3,VEF,2025,2,91.145609,99.865501,A+,34.220694,25.000000,24.848587,7.076328
3,4,D2D,2025,2,90.571975,99.798252,A+,33.561911,24.655012,24.461642,7.893410
4,5,LAI,2024,5,90.170786,99.731002,A+,33.092142,23.518994,23.199865,10.359785
5,6,HGM,2025,2,88.634794,99.663753,A+,34.852796,24.259548,24.882234,4.640215
6,7,AGX,2024,5,88.441908,99.596503,A+,33.377041,23.140359,24.242934,7.681574
7,8,BCE,2025,2,87.292682,99.529254,A+,30.517447,24.402617,23.082100,9.290518
8,9,LBE,2025,2,85.892380,99.462004,A+,30.721368,24.057426,23.401750,7.711836
9,10,TDC,2025,2,85.825117,99.394755,A+,31.501108,24.099746,20.979139,9.245124


In [8]:
out_csv = 'final.csv'
result.to_csv(out_csv, index=False)
print('Saved:', out_csv, 'Rows:', len(result))

print('\nTOP 10:')
print(result.head(10))

print('\nBOTTOM 10:')
print(result.tail(10))


Saved: final.csv Rows: 1487

TOP 10:
   Rank ticker  year  quarter   FA_Score  FA_Percentile FA_Rating  \
0     1    BOT  2025        2  93.923489     100.000000       NaN   
1     2    VNI  2024        5  91.314752      99.932751        A+   
2     3    VEF  2025        2  91.145609      99.865501        A+   
3     4    D2D  2025        2  90.571975      99.798252        A+   
4     5    LAI  2024        5  90.170786      99.731002        A+   
5     6    HGM  2025        2  88.634794      99.663753        A+   
6     7    AGX  2024        5  88.441908      99.596503        A+   
7     8    BCE  2025        2  87.292682      99.529254        A+   
8     9    LBE  2025        2  85.892380      99.462004        A+   
9    10    TDC  2025        2  85.825117      99.394755        A+   

   PROFITABILITY_Score  GROWTH_Score  FINANCIAL_STRENGTH_Score  \
0            33.275843     24.781258                 23.317631   
1            31.823339     24.915876                 23.822342   
2    